In [2]:
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

# CSV 파일 읽기
df = pd.read_csv('Data/seven_latlng2.csv')



In [ ]:
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

# CSV 파일 읽기
df = pd.read_csv('Data/seven_latlng2.csv')

# 주소 정제 함수
def clean_address(address):
    address = re.sub(r'\(.*?\)', '', address)
    address = re.sub(r'[0-9]+호', '', address)
    address = re.sub(r'[0-9]+층', '', address)
    address = re.sub(r'\s+', ' ', address).strip()
    return address

# 주소 정제 적용
df['cleaned_address'] = df['주소'].apply(clean_address)

# Nominatim 초기화
geolocator = Nominatim(user_agent="myGeocoder")

# 지오코딩 함수
def geocode_address(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        print(f"Geocoding request timed out for address: {address}")
        return None, None
    finally:
        time.sleep(10)  # API 사용 제한을 피하기 위해 각 요청 후 1초 대기

# 지오코딩 적용
df['new_lat'], df['new_lng'] = zip(*df['cleaned_address'].apply(geocode_address))

# 결과 저장
df.to_csv('seven.csv', index=False)

# 지오코딩 결과 확인
print(df[['매장명', 'cleaned_address', 'new_lat', 'new_lng']].head())

In [9]:
import pandas as pd
import requests
import time


# 브이월드 API 키
API_KEY = " vworld key "

# 주소 정제 함수
def clean_address(address):
    address = address.replace('서울시', '서울특별시')
    address = re.sub(r'\(.*?\)', '', address)
    address = re.sub(r'[0-9]+층', '', address)
    address = re.sub(r'[0-9]+호', '', address)
    address = re.sub(r'\s+', ' ', address).strip()
    return address

# 지오코딩 함수
def geocode_address(address):
    base_url = "http://api.vworld.kr/req/address"
    params = {
        "service": "address",
        "request": "getCoord",
        "version": "2.0",
        "crs": "epsg:4326",
        "address": address,
        "refine": "true",
        "simple": "false",
        "format": "json",
        "type": "road",
        "key": API_KEY
    }
    try:
        response = requests.get(base_url, params=params)
        result = response.json()
        
        if result['response']['status'] == 'OK':
            x = float(result['response']['result']['point']['x'])
            y = float(result['response']['result']['point']['y'])
            return y, x  # 위도, 경도 순서로 반환
        else:
            print(f"주소를 찾을 수 없습니다: {address}")
            return None, None
    except Exception as e:
        print(f"오류 발생: {e}")
        return None, None

# 주소 정제 적용
df['cleaned_address'] = df['주소'].apply(clean_address)

# 지오코딩 적용
for index, row in df.iterrows():
    address = row['cleaned_address']
    lat, lng = geocode_address(address)
    
    if lat and lng:
        df.at[index, 'new_lat'] = lat
        df.at[index, 'new_lng'] = lng
    else:
        print(f"위도 경도를 찾을 수 없습니다: {address}")
    
    # API 사용량 제한을 고려하여 요청 간 시간 간격 두기
    time.sleep(1)

# 결과 저장
df.to_csv('seven2.csv', index=False, encoding='utf-8-sig')

# 결과 확인
print(df[['매장명', 'cleaned_address', 'new_lat', 'new_lng']].head())

주소를 찾을 수 없습니다: 서울특별시 강남구 개포로109길 9
위도 경도를 찾을 수 없습니다: 서울특별시 강남구 개포로109길 9
주소를 찾을 수 없습니다: 서울특별시 강남구 논현로57길 24,
위도 경도를 찾을 수 없습니다: 서울특별시 강남구 논현로57길 24,
주소를 찾을 수 없습니다: 서울특별시 강남구 남부순환로361길 6
위도 경도를 찾을 수 없습니다: 서울특별시 강남구 남부순환로361길 6
주소를 찾을 수 없습니다: 서울특별시 강남구 선릉로64길 15 , -1
위도 경도를 찾을 수 없습니다: 서울특별시 강남구 선릉로64길 15 , -1
주소를 찾을 수 없습니다: 서울특별시 강남구 광평로19길 15 목련타운 제 상가동
위도 경도를 찾을 수 없습니다: 서울특별시 강남구 광평로19길 15 목련타운 제 상가동
주소를 찾을 수 없습니다: 서울특별시 강남구 언주로98길 37
위도 경도를 찾을 수 없습니다: 서울특별시 강남구 언주로98길 37
주소를 찾을 수 없습니다: 서울특별시 강남구 테헤란로1길 10,
위도 경도를 찾을 수 없습니다: 서울특별시 강남구 테헤란로1길 10,


KeyboardInterrupt: 